In [1]:
import os, random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

"""
    Namin Neural Network.mk3
    -----------------
    conv1 - relu1 - pool1 - bn1
    conv2 - relu2 - pool2 - bn2
    conv3 - relu3 - pool3 - bn3
    affine - softmax

    Parameters
    ----------
    Input Layer(Input Size): 100*100*3
    First Layer: Conv1(3EA, 3*3*3, Strides=3, Padding=1) - ReLU1
                - 34*34*3 - Pool1(2*2, Strides=1) - 33*33*3 - Bn1
    Second Layer: Conv2(6EA, 6*6*3, Strides=1, Padding=VALID) - ReLU2
                - 28*28*6 - Pool2(2*2, Strides=2) - 14*14*6 - Bn2
    Third Layer: Conv3(9EA, 3*3*6, Strides=3, Padding=2) - ReLU3
                - 6*6*9 - Pool3(2*2, Strides=2) - 3*3*9 - Bn3
    Output Layer: Affine(W=3*3*9, B=9) - Output Nodes = 3
 """

'\n    Namin Neural Network.mk3\n    -----------------\n    conv1 - relu1 - pool1 - bn1\n    conv2 - relu2 - pool2 - bn2\n    conv3 - relu3 - pool3 - bn3\n    affine - softmax\n\n    Parameters\n    ----------\n    Input Layer(Input Size): 100*100*3\n    First Layer: Conv1(3EA, 3*3*3, Strides=3, Padding=1) - ReLU1\n                - 34*34*3 - Pool1(2*2, Strides=1) - 33*33*3 - Bn1\n    Second Layer: Conv2(6EA, 6*6*3, Strides=1, Padding=VALID) - ReLU2\n                - 28*28*6 - Pool2(2*2, Strides=2) - 14*14*6 - Bn2\n    Third Layer: Conv3(9EA, 3*3*6, Strides=3, Padding=2) - ReLU3\n                - 6*6*9 - Pool3(2*2, Strides=2) - 3*3*9 - Bn3\n    Output Layer: Affine(W=3*3*9, B=9) - Output Nodes = 3(Frank, Mike, T)\n '

In [2]:
# Load to Images
trainlist, testlist = [], []
with open('train.txt') as f:
    for line in f:
        tmp = line.strip().split()
        trainlist.append([tmp[0], tmp[1]])
        
with open('test.txt') as f:
    for line in f:
        tmp = line.strip().split()
        testlist.append([tmp[0], tmp[1]])

In [3]:
# Image Preprocessing
IMG_H = 100
IMG_W = 100
IMG_C = 3

def readimg(path):
    img = plt.imread(path)
    return img

def batch(path, batch_size):
    img, label, paths = [], [], []
    for i in range(batch_size):
        img.append(readimg(path[0][0]))
        label.append(int(path[0][1]))
        path.append(path.pop(0))
        
    return img, label

In [4]:
# Neural Network
num_class = 3

with tf.Graph().as_default() as g:
    X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C]) # Input Layer
    Y = tf.placeholder(tf.int32, [None])
    
    with tf.variable_scope('CNN'):
        # 1st Layer(Conv1 - relu1 - maxpool1 - bn1) = 33*33*3
        conv1 = tf.layers.conv2d(X, 3, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(conv1, 2, (1, 1), padding='VALID')
        bn1 = tf.compat.v1.layers.batch_normalization(pool1, training=True)
        # 2nd Layer(Conv2 - relu2 - maxpool2 - bn2) = 14*14*6
        conv2 = tf.layers.conv2d(bn1, 6, 6, (1, 1), padding='VALID', activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(conv2, 2, (2, 2), padding='VALID')
        bn2 = tf.compat.v1.layers.batch_normalization(pool2, training=True)
        # 3rd Layer(Conv3 - relu3 - maxpool3 - bn3) = 3*3*9
        conv3 = tf.layers.conv2d(bn2, 9, 3, (3, 3), padding='SAME', activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(conv3, 2, (2, 2), padding='VALID')
        bn3 = tf.compat.v1.layers.batch_normalization(pool3, training=True)
        # Fully Connected Layer(Affine)
        affine1 = tf.layers.flatten(bn3)
        # Output Layer
        output = tf.layers.dense(affine1, num_class)
        
    # Softmax with Loss
    with tf.variable_scope('Loss'):
        Loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= Y, logits=output))
    
    # Training with Adam    
    train_step = tf.train.AdamOptimizer(0.005).minimize(Loss) 
    saver = tf.train.Saver()
    
    tf.summary.scalar('Epoch-Loss', Loss)
    merged = tf.summary.merge_all()

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


In [5]:
# Size
np.sum([np.product(var.shape) for var in g.get_collection('trainable_variables')]).value

1380

In [6]:
# Setting Batch with Training
batch_size = 1261
epoch = 1000

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('tf_board', sess.graph)
    for i in range(epoch):
        batch_data, batch_label = batch(trainlist, batch_size)     
        _, loss, summary = sess.run([train_step, Loss, merged], feed_dict = {X: batch_data, Y: batch_label})
        print("Epoch:",i+1,"Loss:",loss)
        if i % 10 == 0:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)
        elif i+1 == epoch:
            summary_writer.add_summary(summary, i)
            saver.save(sess, 'logs/model.ckpt', global_step = i+1)

Epoch: 1 Loss: 1.3819227
Epoch: 2 Loss: 0.35917968
Epoch: 3 Loss: 0.19393207
Epoch: 4 Loss: 0.16285183
Epoch: 5 Loss: 0.14350526
Epoch: 6 Loss: 0.18016289
Epoch: 7 Loss: 0.096072815
Epoch: 8 Loss: 0.17399748
Epoch: 9 Loss: 0.15621638
Epoch: 10 Loss: 0.07951683
Epoch: 11 Loss: 0.0698328
Epoch: 12 Loss: 0.06981332
Epoch: 13 Loss: 0.25586703
Epoch: 14 Loss: 0.1385676
Epoch: 15 Loss: 0.067475356
Epoch: 16 Loss: 0.041492168
Epoch: 17 Loss: 0.042621274
Epoch: 18 Loss: 0.059281852
Epoch: 19 Loss: 0.05858392
Epoch: 20 Loss: 0.02995291
Epoch: 21 Loss: 0.025730137
Epoch: 22 Loss: 0.020672137
Epoch: 23 Loss: 0.044005826
Epoch: 24 Loss: 0.044503685
Epoch: 25 Loss: 0.026002387
Epoch: 26 Loss: 0.020278605
Epoch: 27 Loss: 0.012168524
Epoch: 28 Loss: 0.01317683
Epoch: 29 Loss: 0.014116745
Epoch: 30 Loss: 0.015585636
Epoch: 31 Loss: 0.019042492
Epoch: 32 Loss: 0.01270023
Epoch: 33 Loss: 0.009202935
Epoch: 34 Loss: 0.007416865
Epoch: 35 Loss: 0.0077478164
Epoch: 36 Loss: 0.0067612333
Epoch: 37 Loss: 0.0

Epoch: 269 Loss: 0.00071099296
Epoch: 270 Loss: 0.00054122467
Epoch: 271 Loss: 0.0003373319
Epoch: 272 Loss: 0.0006072743
Epoch: 273 Loss: 0.00044426555
Epoch: 274 Loss: 0.00043964727
Epoch: 275 Loss: 0.00041954627
Epoch: 276 Loss: 0.0005484054
Epoch: 277 Loss: 0.0005001764
Epoch: 278 Loss: 0.0003150458
Epoch: 279 Loss: 0.0005893107
Epoch: 280 Loss: 0.0005729623
Epoch: 281 Loss: 0.00043880485
Epoch: 282 Loss: 0.0004076848
Epoch: 283 Loss: 0.00031195738
Epoch: 284 Loss: 0.00054916536
Epoch: 285 Loss: 0.0004430061
Epoch: 286 Loss: 0.00056389795
Epoch: 287 Loss: 0.0005547115
Epoch: 288 Loss: 0.00036078697
Epoch: 289 Loss: 0.00039320474
Epoch: 290 Loss: 0.00028419553
Epoch: 291 Loss: 0.0006045909
Epoch: 292 Loss: 0.0005103713
Epoch: 293 Loss: 0.00033674433
Epoch: 294 Loss: 0.00053149316
Epoch: 295 Loss: 0.0003673604
Epoch: 296 Loss: 0.000392704
Epoch: 297 Loss: 0.00036859038
Epoch: 298 Loss: 0.0006115126
Epoch: 299 Loss: 0.00043885424
Epoch: 300 Loss: 0.0002697489
Epoch: 301 Loss: 0.000520

Epoch: 536 Loss: 0.00014602381
Epoch: 537 Loss: 0.00014992495
Epoch: 538 Loss: 0.00014200823
Epoch: 539 Loss: 0.00021133042
Epoch: 540 Loss: 0.00018186077
Epoch: 541 Loss: 0.000108432374
Epoch: 542 Loss: 0.00021230102
Epoch: 543 Loss: 0.00020036132
Epoch: 544 Loss: 0.000153047
Epoch: 545 Loss: 0.00014080739
Epoch: 546 Loss: 0.0001117994
Epoch: 547 Loss: 0.00020278667
Epoch: 548 Loss: 0.00015523385
Epoch: 549 Loss: 0.00020575008
Epoch: 550 Loss: 0.00020120613
Epoch: 551 Loss: 0.0001242602
Epoch: 552 Loss: 0.00013860155
Epoch: 553 Loss: 9.746389e-05
Epoch: 554 Loss: 0.00022391863
Epoch: 555 Loss: 0.00019736166
Epoch: 556 Loss: 0.00012559634
Epoch: 557 Loss: 0.00019812903
Epoch: 558 Loss: 0.00012662368
Epoch: 559 Loss: 0.00014182196
Epoch: 560 Loss: 0.00013212576
Epoch: 561 Loss: 0.0002528096
Epoch: 562 Loss: 0.00016742607
Epoch: 563 Loss: 9.740551e-05
Epoch: 564 Loss: 0.00019708219
Epoch: 565 Loss: 0.0001667519
Epoch: 566 Loss: 0.00014261686
Epoch: 567 Loss: 0.00013207659
Epoch: 568 Loss

Epoch: 804 Loss: 5.4290012e-05
Epoch: 805 Loss: 0.00011126937
Epoch: 806 Loss: 0.00010039801
Epoch: 807 Loss: 7.9192614e-05
Epoch: 808 Loss: 7.185477e-05
Epoch: 809 Loss: 5.8374866e-05
Epoch: 810 Loss: 0.00010545766
Epoch: 811 Loss: 7.746503e-05
Epoch: 812 Loss: 0.00011081501
Epoch: 813 Loss: 0.000105919156
Epoch: 814 Loss: 6.289917e-05
Epoch: 815 Loss: 7.129316e-05
Epoch: 816 Loss: 4.9010243e-05
Epoch: 817 Loss: 0.00011819103
Epoch: 818 Loss: 0.00010494532
Epoch: 819 Loss: 6.797177e-05
Epoch: 820 Loss: 0.00010509223
Epoch: 821 Loss: 6.266612e-05
Epoch: 822 Loss: 7.3912255e-05
Epoch: 823 Loss: 6.8347144e-05
Epoch: 824 Loss: 0.00014434088
Epoch: 825 Loss: 8.8465116e-05
Epoch: 826 Loss: 5.0052753e-05
Epoch: 827 Loss: 0.00010511539
Epoch: 828 Loss: 8.415709e-05
Epoch: 829 Loss: 7.462349e-05
Epoch: 830 Loss: 6.930943e-05
Epoch: 831 Loss: 6.2683706e-05
Epoch: 832 Loss: 9.644278e-05
Epoch: 833 Loss: 6.320613e-05
Epoch: 834 Loss: 0.00010706398
Epoch: 835 Loss: 0.0001022428
Epoch: 836 Loss: 6.

In [8]:
# Print an Accuracy
acc = 0

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(len(testlist)):
        batch_data, batch_label = batch(testlist, 1)
        logit = sess.run(output, feed_dict = {X:batch_data})
        if np.argmax(logit[0]) == batch_label[0]:
            acc += 1
        else:
            print(logit[0], batch_label[0])
            
    accuracy = acc/len(testlist)
    print("Accuracy:", accuracy)

INFO:tensorflow:Restoring parameters from logs\model.ckpt-1000
[-0.00776708  1.0595394  -1.8102638 ] 0
[-0.08547989  1.3054487  -1.6776209 ] 0
[-0.03934199  1.2908739  -1.863743  ] 0
[ 0.280813   1.1099066 -1.6594608] 0
[ 0.12458479  0.5998282  -1.8284099 ] 0
[ 0.36861125  0.9803136  -1.4008007 ] 0
[ 0.49296266  0.8132829  -1.3991555 ] 0
[ 0.22975308  0.6291299  -1.7825828 ] 0
[-0.44606525  0.21892989  1.2427061 ] 1
[-1.7054174  1.6543214  1.8001354] 1
[-0.745101    0.63360816  1.7450112 ] 1
[ 0.7857095   0.6305351  -0.03069509] 1
Accuracy: 0.9755102040816327


In [ ]:
# Print an Answer
